Instalar *boto3* (*AWS SDK* para *Python*)

In [1]:
!pip install boto3

In [1]:
import boto3
import json
import os
import time
from boto3.dynamodb.conditions import Key

region = 'us-east-1'

In [2]:
rek = boto3.client('rekognition', region_name=region)

Crear uan colección para almacenar los *faceId* de los usuarios registrados. Sobre esta colección es sobre la que se harán consultas en la operación de reconocimiento facial

In [58]:
face_collection = 'big-eye-muii-face-collection'
rek.create_collection(CollectionId=face_collection)

In [3]:
response = rek.list_collections(MaxResults=10)
response

Comprobamos que no haya nigún *faceId* almacenado por el momento

In [5]:
response = rek.list_faces(CollectionId=face_collection)
response

En caso de que no haya, para probar la función de renocimiento debemos añadir una. Para ello, tenemos que subir una imagen a un *Bucket* de *S3*.

In [ ]:
bucket_name = 'bucket-name'
object_key = 'object-key-on-bucket'

response = rek.index_faces(
    CollectionId='big-eye-muii-face-collection',
    Image={
        'S3Object':{
            'Bucket': bucket_name,
            'Name': object_key
        }
    },
    ExternalImageId= object_key,
    MaxFaces=1,
    QualityFilter="AUTO",
    DetectionAttributes=['ALL']
)

faceId = response['FaceRecords'][0]['Face']['FaceId']

Además, para que la función se ejecute correctamente hemos de simular que el usuario anterior se ha registrado añadiendo una entrada a la tabla de usuarios.

In [ ]:
user_table = 'USER_TABLE'

dyn_resource = boto3.resource('dynamodb', region_name=region)
dyn_client = boto3.client('dynamodb', region_name=region)

tables = dyn_client.list_tables()['TableNames']

if user_table not in tables:
    dyn_resource.create_table(
        TableName=user_table,
        KeySchema=[
            {
                'AttributeName': 'faceId',
                'KeyType': 'HASH'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'faceId'
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'fullName',
                'AttributeType': 'S'
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )
    
user_table_res = dyn_resource.Table(user_table)
user_table_res.put_object(Item={
    'faceId': faceId,
    'fullName' 'user-name'
})

Además, si no está creada, también se debe crear la tabla de logs

In [ ]:
log_table = 'LOG_TABLE'

if log_table not in tables:
    dyn_resource.create_table(
        TableName=log_table,
        KeySchema=[
            {
                'AttributeName': 'faceId',
                'KeyType': 'HASH'
            },
            {
                'AttributeName': 'ts',
                'KeyType': 'RANGE'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'faceId'
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'ts',
                'KeyType': 'N'
            },
            {
                'AttributeName': 'fullName',
                'AttributeType': 'S'
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )

Por último, ejecutamos la lógica de la función *lambda*. Cabe destacar que a la función desplegada habrá que hacerle una serie de cambios:
* El nombre del *bucket* y el nombre del fichero serán establecidos a partir del evento desencadenador (parámetro *event*)
* Los nombres de las tablas y la coleción de *Rekognition* serán establecidos como variables de entorno y extraídos mediante *os.environ*

In [1]:
def lambda_handler(event, context):
    # In case the collections does not exists, return HTTP404 NOT FOUND
    if face_collection not in rek.list_collections()['CollectionIds']:
        print(f"Collection with id {face_collection} not found")

    response = rek.search_faces_by_image(
        CollectionId=face_collection,
        Image={
            'S3Object':{
                'Bucket': bucket_name,
                'Name': object_key
            }
        },
        MaxFaces=1
    )

    if not response['FaceMatches']:
        # In case no face with similarity above 90% was found on collections, retunr HTTP403 FORBIDDEN
        print("Face not registered with threshold above 80%")
    else:
        # Otherwise, return HTTP200 SUCCESSFULL and increment entry on log table
        faceMatch = response['FaceMatches'][0]

        dyn_log_table = dyn_resource.Table('backend-muii-metadata')

        print(faceMatch['Face']['FaceId'])
        response = user_table_res.query(KeyConditionExpression=Key('faceId').eq(faceMatch['Face']['FaceId']))
        userInfo = response.get('Items', [])

        if not userInfo:
            print("User not registered")
        
        user_log_info = {
            'faceId': faceMatch['Face']['FaceId'],
            'ts': time.time_ns(),
            'fullName': userInfo[0]['fullName']
        }

        dyn_log_table.put_item(Item=user_log_info)

        print(f"FaceId [{faceMatch['Face']['FaceId']} was found with {faceMatch['Similarity']:.2f}%]")

Ya que no necesitamos extraer nada del evento desencadenador, podemos ejecutar la función sin parámetros para comprobar la salida

In [32]:
lambda_handler(None, None)

4d55d042-f5af-4574-9ed9-00387702e987
FaceId [4d55d042-f5af-4574-9ed9-00387702e987 was found with 100.00%]
